# Sincronizar CAT y lista local de agrobd

In [1]:
import json
import pandas as pd
import requests

# local import
from paths import *

In [2]:
agrobd_list = pd.read_csv('../tests/test_sync_input-agrobd.csv', keep_default_na=False)

In [3]:
def getInfoTaxon(record_id):
    '''
    Hace una consulta a zacatuche para obtener la información que se necesita de un id.
    Devuelve una pandas.Series con la información solicitada en la query.
    '''
    query = """query taxon{
                dwcTaxon(taxonID:"""+"\""+record_id+"\""+"""){
                            id
                            taxonomicStatus
                            scientificName
                            acceptedNameUsage{
                            id
                            scientificName
                            }
                        }
                        }"""
    
    # renombrar las columnas
    New_col_names = {'acceptedNameUsage.id': 'id_valido',
                     'acceptedNameUsage.scientificName' : 'taxon_valido',
                     'taxonomicStatus': 'estatus',
                     'scientificName': 'taxon'
                    }

    path_zacatuche = 'http://zacatuche2.conabio.gob.mx:4000/graphql'
    r = requests.post(path_zacatuche, json={'query': query})

    # TO DO: check that response.status = 200, else print error to log

    json_data = json.loads(r.text)

    # case when id is not in CAT
    if json_data['data']['dwcTaxon'] is None:
        return None

    # case when there is no id_valido associated to id
    if json_data['data']['dwcTaxon']['acceptedNameUsage'] is None:
        df_data = (pd.json_normalize(json_data['data']['dwcTaxon'])
                     .rename(columns = New_col_names))
        df_data[['id_valido', 'taxon_valido']] = None, None
        return df_data.loc[0]
    
    df_data = pd.json_normalize(json_data['data']['dwcTaxon'])
    df_data = (df_data.rename(columns = New_col_names)
                          .fillna(''))
    
    return df_data.loc[0]

In [4]:
def getInfoTaxon(record_id):
    zacatuche = pd.read_csv('../tests/test_sync_input-zacatuche.csv', keep_default_na=False)
    try:
        res = zacatuche.loc[zacatuche['id'] == record_id]
        res = res.reset_index(drop=True)
        res = res.loc[0]
    except KeyError:
        res = None
    return res

In [5]:
getInfoTaxon('177_pendiente')

id                                          177_pendiente
taxon                                       Citrus maxima
estatus                                                  
id_valido                                                
taxon_valido                                             
comentarios_revision    Prueba: sin cambios, id pendiente
Name: 0, dtype: object

In [6]:
def updateLocal(agrobd_id, New_values):
    '''
    Realiza las modificaciones en la instancia de catalogo-agrobiodiversidad
    de los campos que se le pasen como parámetro.
    '''
    New_values['usuario'] = 'Bot validación'
    
    # ¿Cómo deben aparecer los valores vacíos? 
    new_values = ''  
    for field in New_values:
        new_values += f'{field}: \"{New_values[field]}\" \n'

    query = f'''mutation{{
                updateAgrobiodiversidad(
                    id:"{agrobd_id}"
                    {new_values}){{
                    id
                }}
                }}'''
    
    # requests.post(paths.siagro, json={'query': query})
    test_updateLocal(agrobd_id, New_values)


In [7]:
def test_updateLocal(agrobd_id, New_values):
    result = agrobd_list.loc[agrobd_list['id'] == agrobd_id].copy()
    for field in New_values:
        result.loc[result['id']==agrobd_id, field] = New_values[field]
    print(f'\n updateLocal: \n')
    print(result.reset_index(drop=True).loc[0])
    print(result['comentarios_revision'].values)

In [8]:
def is_synonym(record):  
    return (record['id_valido']) and (record['id'] != record['id_valido'])

In [9]:
def request_agrobd_review(record, check_previous_label=True):
    # if record doesn't have agrobd label, request review
    check = True
    if check_previous_label:
        check = record['categoria_agrobiodiversidad'] != 'Agrobiodiversidad'

    if check:
        note = record['comentarios_revision'] + '\n REVISAR ETIQUETA AGROBIODIVERSIDAD'
        updateLocal(record['id'], {'comentarios_revision': note})

In [10]:
def delete_agrobd_label(record):
    # TO DO: also delete subcategoria_agrobiodiversidad
    # TO DO: cómo poner campos vacíos    
    updateLocal(record['id'], {'categoria_agrobiodiversidad': ''})

In [11]:
def id_is_in_agrobd_list(record_id):
    return record_id in agrobd_list['id'].values

In [12]:
def add_record_to_agrobd_list(record):
    query = f'''mutation{{
                addAgrobiodiversidad(
                    id:"{record['id']}"
                    taxon:\"{record['taxon']}\"
                    estatus:\"{record['estatus']}\"
                    id_valido:"{record['id_valido']}"
                    taxon_valido:\"{record['taxon_valido']}\"
                    usuario:\"Bot validación\"){{
                    id
                }}
                }}'''
    #requests.post(paths.siagro, json={'query': query})
    test_add_record(record)

In [13]:
def test_add_record(record):
    # Note: `usario: Bot validación` is missing from this test
    temp = pd.DataFrame(record)
    agrobd_list.loc[record['id'], :] = record
    print(f'\n add_record_to_agrobd_list: \n {record}')

In [14]:
def add_agrobd_label(record_id, agrobd):
    ''' 
    Agrega la etiqueta "Agrobiodiversidad" a un registro.
    También agrega la subcategoría y las referencia que hereda del registro 
    cuyo estatus cambió de válido --> sinónimo.
    
    TO DO: Decidir qué hacer en caso de que ya tenga la etiqueta y referencias.

    Recibe:
        record_id (string): nuevo id_valido del registro agrobd
        agrobd (pandas.Series): registro cuyo estatus cambio de válido -> sinónimo
    '''
    updateLocal(record_id, {'categoria_agrobiodiversidad': 'Agrobiodiversidad',
                            'subcategoria_agrobiodiversidad': agrobd['subcategoria_agrobiodiversidad'],
                            'referencia': agrobd['referencia']})

In [15]:
def sync_status_and_agrobd_label(agrobd, catalog):
    '''
    Esta función actualiza los campos de estatus, id_valido, taxon_valido y
    categoria_agrobiodiversidad en la lista local si el registro `agrobd`
    tuvo un cambio de estatus en CAT. Los cambios dependen del tipo de 
    transición que ocurrió para el estatus del registro. Ver detalles en README. 

    Recibe:
        agrobd (pandas.Series): Un registro de la lista local de agrobiodiversidad
        catalog (pandas.Series): Versión de CAT Conabio del registro con mismo id que agrobd
    '''
    # sync estatus, id_valido, taxon_valido
    updateLocal(agrobd['id'], {'estatus': catalog['estatus'],
                               'id_valido': catalog['id_valido'],
                               'taxon_valido': catalog['taxon_valido']
                              })

    # sinonimo --> valido OR NA --> valido
    if agrobd['id'] == catalog['id_valido']:
        request_agrobd_review(agrobd)
    
    # valido --> NA
    elif (agrobd['id'] == agrobd['id_valido']) and (not catalog['id_valido']):
        delete_agrobd_label(agrobd) # TO DO: preguntar a CAT qué hacer en este caso
    
    # sinonimo --> sinonimo
    elif is_synonym(agrobd) and is_synonym(catalog):
        if not id_is_in_agrobd_list(catalog['id_valido']):
            new_record = getInfoTaxon(catalog['id_valido'])
            add_record_to_agrobd_list(new_record)
            request_agrobd_review(new_record, check_previous_label=False)
    
    # valido --> sinonimo
    elif (agrobd['id'] == agrobd['id_valido']) and is_synonym(catalog):
        delete_agrobd_label(agrobd)
        if not id_is_in_agrobd_list(catalog['id_valido']):
            new_record = getInfoTaxon(catalog['id_valido']) # assumes id exists in CAT
            add_record_to_agrobd_list(new_record)
        # this is the only line that adds the agrobd label to a record
        add_agrobd_label(catalog['id_valido'], agrobd)
    

In [16]:
def sync_agrobd_to_catalog(agrobd_list):
    ''' Pipeline para sincronizar la lista local de agrobiodiversidad
    con el catálogo de Conabio (CAT). El script recorre la lista registro por registro;
    compara la versión actual de la lista `agrobd_list` contra CAT mediante una consulta
    a zacatuche. Si cambió el estatus de un registro en CAT actualiza los campos
    y realiza los cambios necesarios en la categoría agrobiodiversidad.
    Envía un correo para advertir de ids que no están en CAT, pues se trata
    de un error al que se le debe dar seguimiento.
    
    Recibe:
        agrobd_list(pandas.DataFrame): Tabla que contiene la versión más actual
        de la lista local de agrobd. 
    '''
    sin_taxon_valido_asociado = []

    for i, agrobd in agrobd_list.iterrows():
        print('\n ', str(agrobd['comentarios_revision']).upper())
        if 'pendiente' in agrobd['id']:
            continue

        catalog = getInfoTaxon(agrobd['id'])

        # case when agrobd is not in CAT
        if catalog is None:
            sin_taxon_valido_asociado.append((agrobd['id'], agrobd['taxon']))
            continue
        
        # case when there was a change in taxonomic status

        if agrobd['id_valido'] != catalog['id_valido']:
            sync_status_and_agrobd_label(agrobd, catalog)

        # case when there was a change in taxonomic name
        if agrobd['taxon'] != catalog['taxon']:
            updateLocal(agrobd['id'], {'taxon': catalog['taxon']})
    
    # format text for email
    # TO DO: align ids and taxon to fit in a table
    email = ''
    for agrobd_id, taxon in sin_taxon_valido_asociado:
        email += f'{agrobd_id} |  {taxon} \n'

    # sendeMail(email)
    print(f'Correo para enviar: {email}')

In [20]:
sync_agrobd_to_catalog(agrobd_list)


  PRUEBA: SIN CAMBIOS

  PRUEBA: SÓLO CAMBIA TAXON

 updateLocal: 

id                                                                        7914ANGIO
taxon                                                   Nopalea karwinskiana prueba
estatus                                                             Aceptado/Válido
id_valido                                                                 7914ANGIO
taxon_valido                                                   Nopalea karwinskiana
referencia                        "Goettsch, Bárbara, Tania Urquiza-Haas, Patric...
categoria_agrobiodiversidad                                       Agrobiodiversidad
subcategoria_agrobiodiversidad                                                     
justificacion_subcategoria                                                         
comentarios_revision                                      Prueba: Sólo cambia taxon
usuario                                                              Bot validación
Name: 0

In [18]:
getInfoTaxon('7914ANGIO')

id                                        7914ANGIO
taxon                   Nopalea karwinskiana prueba
estatus                             Aceptado/Válido
id_valido                                 7914ANGIO
taxon_valido                   Nopalea karwinskiana
comentarios_revision      Prueba: Sólo cambia taxon
Name: 0, dtype: object

In [21]:
agrobd_list.tail()

,id,taxon,estatus,id_valido,taxon_valido,referencia,categoria_agrobiodiversidad,subcategoria_agrobiodiversidad,justificacion_subcategoria,comentarios_revision,usuario
11,75732ANGIO,Rubus aboriginum,Sinónimo,7908ANGIO,Nopalea nuda,"""Goettsch, Bárbara, Tania Urquiza-Haas, Patric...",,,,"Prueba: Estatus sinónimo -> sinónimo, nuevo id...",admin@zen.dro
12,76752ANGIO,Solanum palmeri,Aceptado/Válido,76752ANGIO,Solanum palmeri,"""Goettsch, Bárbara, Tania Urquiza-Haas, Patric...",Agrobiodiversidad,,,"Prueba: Estatus Válido -> Sinónimo, nuevo id_v...",admin@zen.dro
13,6206PECES,Canthigaster rostrata,Aceptado/Válido,6206PECES,Canthigaster rostrata,El taxón pertenece a la categoría de Consumo h...,Agrobiodiversidad,,,"Prueba: Estatus Válido -> Sinónimo, nuevo id_v...",admin@zen.dro
75735ANGIO,75735ANGIO,Rubus flagellaris,Aceptado/Válido,75735ANGIO,Rubus flagellaris,NaN,NaN,NaN,NaN,,NaN
55316ANGIO,55316ANGIO,Solanum (Solanum) (Dulcamara) umbelliferum,Aceptado/Válido,55316ANGIO,Solanum (Solanum) (Dulcamara) umbelliferum,NaN,NaN,NaN,NaN,,NaN


Ayuda en:
* Probar formato para el texto del correo.
* Cómo representar valores vacíos en mutations para zacatuche
* queries y mutations para zacatuche
* Que el código represente lo que está en el diagrama
* Que el diagrama represente todas las acciones que queremos que haga el script